In [204]:
import numpy as np
import helper_functions as hf
high = np.load('pcg/P009_S001_002.npy')
low = np.load('pcg/P017_S002_001.npy')

#import helper_functions as hf
pcg_dict = {'low': low, 'high': high}
d = hf.resample_and_normalize(pcg_dict=pcg_dict, from_freq=22050, to_freq=1000)

In [205]:
import numpy as np
from kymatio import Scattering1D

J = 4
Q = 12
fs = 1000
sec = 30
L = int(np.floor(fs*sec/2**J))*2**J

i = 3
S = Scattering1D(J=J, shape=L, Q=Q, max_order=2)

low, high = d['low'][L*i:L*(i+1)], d['high'][L*i:L*(i+1)]
print(L/2**J)
Sx_low = S(low)
Sx_high = S(high)
print(Sx_low.shape, Sx_high.shape)

meta = S.meta()
order0 = np.where(meta['order'] == 0)
order1 = np.where(meta['order'] == 1)
order2 = np.where(meta['order'] == 2)

print(Sx_low[order0].shape, Sx_low[order1].shape, Sx_low[order2].shape)
print(order1)

1875.0
(65, 1875) (65, 1875)
(1, 1875) (28, 1875) (36, 1875)
(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28], dtype=int64),)


In [206]:
scat1d = Scattering1D(J=J, shape=L, Q=Q, max_order=2)
meta = scat1d.meta()
sr = 1000
freq_min = 0

# carrier center frequencies condition >= freq_min
freq_cnd = meta['xi'][:, 0] * sr >= freq_min

# indices for scat coefficient to extract 
idx_S1 = np.where(np.logical_and(meta['order'] == 1, freq_cnd))[0]  # order 1
idx_S2 = np.where(np.logical_and(meta['order'] == 2, freq_cnd))[0] # order 2
idx_S1S2 = np.where(np.logical_and(meta['order'] != 0, freq_cnd))[0] # order 1 and 2

In [207]:
xi1s = np.array([sr * psi['xi'] for psi in scat1d.psi1_f])
xi2s = np.array([sr * psi['xi'] for psi in scat1d.psi2_f])

sigma1s = [sr * psi['sigma'] for psi in scat1d.psi1_f]
sigma2s = [sr * psi['sigma'] for psi in scat1d.psi2_f]

mid_xi1s = np.sqrt(xi1s[1:] * xi1s[:-1])  # center for two valuess in a log scale
mid_xi2s = np.sqrt(xi2s[1:] * xi2s[:-1])
x_coords = np.array([sr//2] + list(mid_xi2s) + [xi2s[-1]-sigma2s[-1]])
y_coords = np.array([sr//2] + list(mid_xi1s) + [xi1s[-1]-sigma1s[-1]])

x_coords = np.hstack((x_coords, x_coords[-1] / 2))

In [208]:
# NORMALIZED SCATTERING TRANSFORM

def make_parent_daughter_index_pairs(meta):
    ks = meta['key']
    pairs = []
    parents = []
    for i, k in enumerate(ks):
        if len(k) == 1:
            parents.append([i, k[0]])
    for i, k in enumerate(ks):
        if len(k) == 2:
            for ip, parent in enumerate(parents):
                if parent[1] == k[0]:
                    pairs.append([ip+1, i])
    return pairs


def normalize_scattering_data(Sx_in, meta, x):
    """
    This function normalizes scattering data by dividing first order coefficient by the average absolute value of x over L
    and each second order coefficient by the value of its first order parent.
    
    :param Sx: a numpy array containing the scattered data from temporal scattering
    :param meta: A dictionary containing metadata about the scattering 
    :param x: the original PCG signal
    :return: The function `normalize_scattering_data` returns the normalized scattering data `Sx_norm`.
    """
    Sx = Sx_in.copy()
    pairs = make_parent_daughter_index_pairs(meta)
    Sx_norm = np.zeros_like(Sx)
    S1_max = len(Sx[meta['order'] == 1])+1
    Sx_norm = Sx
    x_abs_avg = np.average(abs(x), axis=0) # average abs(x) over time (L)
    print(Sx.shape)
    for i in range(1,S1_max):
        for j in range(Sx.shape[1]):
            Sx_norm[i,j] = Sx[i,j] / x_abs_avg #S_1 normalization
    for parent, daughter in pairs:
        for j in range(Sx.shape[1]):
            Sx_norm[daughter,j] = Sx[daughter,j]/Sx[parent,j] #S_2 normalization
    return Sx_norm
    
Sx_low_norm = normalize_scattering_data(Sx_low, S.meta(), low)
Sx_high_norm = normalize_scattering_data(Sx_high, S.meta(), high)

(65, 1875)
(65, 1875)


In [209]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib qt

plt.plot(np.arange(Sx_low[1:,0].shape[0]), Sx_low[1:,0])
plt.title(r'$Sx$', fontsize=16)
plt.ylabel('Value', fontsize=14)
plt.xlabel('Scattering Coefficient Index', fontsize=14)
plt.show()



In [210]:
plt.plot(np.arange(Sx_low_norm[1:,0].shape[0]), Sx_low_norm[1:,0])
plt.title(r'$\widetilde{S}x$', fontsize=16)
plt.ylabel('Value', fontsize=14)
plt.xlabel('Scattering Coefficient Index', fontsize=14)
plt.show()

In [211]:
def remove_bad_features(J, Q, L, features, sr, keep_5060100hz=False, thres=7):
    """
    This function removes bad features from a set of features based on the frequency of the underlying 
    wavelet and returns the remaining features along with their corresponding frequency values.
    
    :param J: The scale of the scattering transform
    :param Q: Q is the number of wavelets per octave used in the scattering transform. It determines the
    frequency resolution of the transform
    :param L: The length of the input signal
    :param features: A numpy array of shape (L, num_features) containing the scattering coefficients of
    an audio signal
    :param sr: sampling rate of the audio signal
    :param keep_5060100hz: A boolean parameter that determines whether to keep the bad frequencies of
    50Hz, 60Hz, and 100Hz or not. If set to True, these frequencies will be kept and not considered as
    bad frequencies. If set to False, all the bad frequencies will be removed including these three,
    defaults to False (optional)
    :param thres: The threshold value used to determine if a feature should be removed based on its
    proximity to a bad frequency. If the distance between the feature's center frequency and a bad
    frequency is less than or equal to thres, the feature is considered bad and will be removed,
    defaults to 7 (optional)
    :return: a tuple containing four elements:
    1. A numpy array of new features after removing the bad features
    2. A list of the new features corresponding first order wavelet center frequencies
    3. A list of the new features corresponding second order wavelet center frequencies
    4. A list of the original features corresponding first order wavelet center frequencies
    5. A list of the original features corresponding second order wavelet center frequencies
    """
    bad_freqs = [50, 60, 100, 120, 150, 180, 200, 240,250,300,350,360,400,420,450,480,500]
    if keep_5060100hz: bad_freqs = bad_freqs[3:]
    S = Scattering1D(J=J, shape=L, Q=Q, max_order=2)
    xi1s = S.meta()['xi'][:,0]*sr
    xi2s = S.meta()['xi'][:,1]*sr
    order1 = np.where(meta['order'] == 1)[0]
    order2 = np.where(meta['order'] == 2)[0]
    print(order1)
    print(S.meta()['xi'].shape)
    print(len(xi1s))
    print(xi1s)
    print(xi2s)
    sigma1s = S.meta()['sigma'][:,0]*sr
    sigma2s = S.meta()['sigma'][:,0]*sr
    bad_feats = []
    new_feats = []
    new_feat_xi1s = []
    new_feat_xi2s = []
    for i in range(len(xi1s)):
        if i==0: bad_feats.append(i)
        for freq in bad_freqs:
            if (xi1s[i] < freq+sigma1s[i]/2+thres and xi1s[i] > freq-sigma1s[i]/2-thres):
                bad_feats.append(i)
    print(bad_feats)
    for i in range(features.shape[0]):
        if i not in bad_feats:
            new_feats.append(features[i].reshape(-1,1))
            if i in order1:
                new_feat_xi1s.append(i)
            if i in order2:
                new_feat_xi2s.append(i)
    return np.hstack(new_feats).T, new_feat_xi1s, new_feat_xi2s, order1, order2

In [212]:
Sx_low_r, xi1s,xi2s, order1, order2 = remove_bad_features(J, Q, L, sr=1000, features=Sx_low_norm)
Sx_high_r,_,_,_,_= remove_bad_features(J,Q,L,sr=1000,features=Sx_high_norm)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28]
(65, 2)
65
[         nan 456.78638314 431.14893343 406.9504032  384.11003212
 362.55159256 342.20313524 322.99674907 304.86833453 287.75738971
 271.60680843 256.36268963 241.97415747 228.39319157 215.57446671
 203.4752016  192.05501606 181.27579628 166.16947992 151.06316357
 135.95684721 120.85053085 105.7442145   90.63789814  75.53158178
  60.42526543  45.31894907  30.21263271  15.10631636 456.78638314
 456.78638314 431.14893343 431.14893343 406.9504032  406.9504032
 384.11003212 384.11003212 362.55159256 362.55159256 342.20313524
 342.20313524 322.99674907 322.99674907 304.86833453 304.86833453
 287.75738971 287.75738971 271.60680843 271.60680843 256.36268963
 256.36268963 241.97415747 241.97415747 228.39319157 228.39319157
 215.57446671 215.57446671 203.4752016  192.05501606 181.27579628
 166.16947992 151.06316357 135.95684721 120.85053085 105.7442145 ]
[   nan    nan    nan    nan    nan    nan  

In [213]:
print(order1)
print(S.meta()['xi'][order1]*1000)
print(xi1s)
print(S.meta()['xi'][xi1s]*1000)
print(order2)
print(S.meta()['xi'][order2]*1000)
print(xi2s)
print(S.meta()['xi'][xi2s]*1000)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28]
[[456.78638314          nan]
 [431.14893343          nan]
 [406.9504032           nan]
 [384.11003212          nan]
 [362.55159256          nan]
 [342.20313524          nan]
 [322.99674907          nan]
 [304.86833453          nan]
 [287.75738971          nan]
 [271.60680843          nan]
 [256.36268963          nan]
 [241.97415747          nan]
 [228.39319157          nan]
 [215.57446671          nan]
 [203.4752016           nan]
 [192.05501606          nan]
 [181.27579628          nan]
 [166.16947992          nan]
 [151.06316357          nan]
 [135.95684721          nan]
 [120.85053085          nan]
 [105.7442145           nan]
 [ 90.63789814          nan]
 [ 75.53158178          nan]
 [ 60.42526543          nan]
 [ 45.31894907          nan]
 [ 30.21263271          nan]
 [ 15.10631636          nan]]
[4, 7, 9, 10, 13, 14, 18, 20, 24, 27, 28]
[[384.11003212          nan]
 [322.99674907          nan]


In [214]:
# STAI3030
import matplotlib.pyplot as plt
%matplotlib qt


fig = plt.figure(figsize=(10, 10))
plt.subplot(5, 2, 1)
plt.plot(low)
plt.title('Normalized PCG, Low Stress')
plt.xlim(0, len(low))
plt.xlabel(r'$L$')
plt.ylabel(r'$Amplitude$')

plt.subplot(5, 2, 2)
plt.plot(high)
plt.title('Normalized PCG, High Stress')
plt.xlim(0, len(high))
plt.xlabel(r'$L$')
plt.ylabel(r'$Amplitude$')

plt.subplot(5, 2, 3)
plt.imshow(Sx_low_norm[order1], aspect='auto', cmap='jet')
plt.title(r'$\tilde{S}_1$, Low Stress')
plt.xlabel(r'$L/T$')
plt.ylabel(r'$\lambda1$')

plt.subplot(5, 2, 4)
plt.imshow(Sx_high_norm[order1], aspect='auto', cmap='jet')
plt.title(r'$\tilde{S}_1$, High Stress')
plt.xlabel(r'$L/T$')
plt.ylabel(r'$\lambda1$')

plt.subplot(5, 2, 5)
plt.imshow(Sx_low_norm[order2], aspect='auto', cmap='jet')
plt.title(r'$\tilde{S}_2$, Low Stress')
plt.xlabel(r'$L/T$')
plt.ylabel(r'$\lambda1, \lambda_2$')

plt.subplot(5, 2, 6)
plt.imshow(Sx_high_norm[order2], aspect='auto', cmap='jet')
plt.title(r'$\tilde{S}_2$, High Stress')
plt.xlabel(r'$L/T$')
plt.ylabel(r'$\lambda1, \lambda_2$')

plt.subplot(5, 2, 7)
plt.imshow(Sx_low_norm[xi1s], aspect='auto', cmap='jet')
plt.title(r'$\tilde{S}_1$ after feature exclusion, Low Stress')
plt.xlabel(r'$L/T$')
plt.ylabel(r'$\lambda1$')

plt.subplot(5, 2, 8)
plt.imshow(Sx_high_norm[xi1s], aspect='auto', cmap='jet')
plt.title(r'$\tilde{S}_1$ after feature exclusion, High Stress')
plt.xlabel(r'$L/T$')
plt.ylabel(r'$\lambda1$')

plt.subplot(5, 2, 9)
plt.imshow(Sx_low_norm[xi2s], aspect='auto', cmap='jet')
plt.title(r'$\tilde{S}_2$ after feature exclusion, Low Stress')
plt.xlabel(r'$L/T$')
plt.ylabel(r'$\lambda1, \lambda_2$')

plt.subplot(5, 2, 10)
plt.imshow(Sx_high_norm[xi2s], aspect='auto', cmap='jet')
plt.title(r'$\tilde{S}_2$ after feature exclusion, High Stress')
plt.xlabel(r'$L/T$')
plt.ylabel(r'$\lambda1, \lambda_2$')

plt.tight_layout()
plt.show()


In [215]:
import matplotlib.pyplot as plt
%matplotlib qt


fig = plt.figure(figsize=(5, 10))
plt.subplot(2, 2, 1)
plt.imshow(Sx_low_norm[xi1s], aspect='auto', cmap='jet')
plt.axhline(y=2, color='green', linestyle='--', label='287.76 Hz')
plt.axhline(y=3, color='r', linestyle='--', label='271.61 Hz')
plt.title(r'$\tilde{S}_1$ after feature exclusion, Low Stress')
plt.xlabel(r'$L/T$')
plt.ylabel(r'$\lambda1$')
plt.legend()

plt.subplot(2, 2, 2)
plt.imshow(Sx_high_norm[xi1s], aspect='auto', cmap='jet')
plt.axhline(y=2, color='green', linestyle='--', label='287.76 Hz')
plt.axhline(y=3, color='r', linestyle='--', label='271.61 Hz')
plt.title(r'$\tilde{S}_1$ after feature exclusion, High Stress')
plt.xlabel(r'$L/T$')
plt.ylabel(r'$\lambda1$')
plt.legend()

plt.subplot(2, 2, 3)
plt.imshow(Sx_low_norm[xi2s], aspect='auto', cmap='jet')
plt.axhline(y=1, color='green', linestyle='--', label='(384.11, 43.75) Hz')
plt.axhline(y=7, color='r', linestyle='--', label='(271.91, 43.75) Hz')
plt.title(r'$\tilde{S}_2$ after feature exclusion, Low Stress')
plt.xlabel(r'$L/T$')
plt.ylabel(r'$\lambda1, \lambda_2$')
plt.legend()

plt.subplot(2, 2, 4)
plt.imshow(Sx_high_norm[xi2s], aspect='auto', cmap='jet')
plt.axhline(y=1, color='green', linestyle='--', label='(384.11, 43.75) Hz')
plt.axhline(y=7, color='r', linestyle='--', label='(271.91, 43.75) Hz')
plt.title(r'$\tilde{S}_2$ after feature exclusion, High Stress')
plt.xlabel(r'$L/T$')
plt.ylabel(r'$\lambda1, \lambda_2$')
plt.legend()

plt.tight_layout()
plt.show()

In [216]:
import numpy as np
from kymatio import Scattering1D

J = 3
Q = 12
fs = 1000
sec = 10
L = int(np.floor(fs*sec/2**J))*2**J

i = 3
S = Scattering1D(J=J, shape=L, Q=Q, max_order=2)

low, high = d['low'][L*i:L*(i+1)], d['high'][L*i:L*(i+1)]
print(L, len(low))
Sx_low = S(low)
Sx_high = S(high)
print(Sx_low.shape, Sx_high.shape)

meta = S.meta()
order0 = np.where(meta['order'] == 0)
order1 = np.where(meta['order'] == 1)
order2 = np.where(meta['order'] == 2)

print(Sx_low[order0].shape, Sx_low[order1].shape, Sx_low[order2].shape)
print(order1)

10000 10000
(27, 1250) (27, 1250)
(1, 1250) (16, 1250) (10, 1250)
(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16],
      dtype=int64),)


In [217]:
# scattering1d config
kwargs = {
    'shape': 2**14,
    'J': 3,
    'T': 2**3,
    'Q': (12, 1)}

scat1d = Scattering1D(**kwargs)
meta = scat1d.meta()
sr = 1000
freq_min = 0

# carrier center frequencies condition >= freq_min
freq_cnd = meta['xi'][:, 0] * sr >= freq_min

# indices for scat coefficient to extract 
idx_S1 = np.where(np.logical_and(meta['order'] == 1, freq_cnd))[0]  # order 1
idx_S2 = np.where(np.logical_and(meta['order'] == 2, freq_cnd))[0] # order 2
idx_S1S2 = np.where(np.logical_and(meta['order'] != 0, freq_cnd))[0] # order 1 and 2

In [218]:
xi1s = np.array([sr * psi['xi'] for psi in scat1d.psi1_f])
xi2s = np.array([sr * psi['xi'] for psi in scat1d.psi2_f])

sigma1s = [sr * psi['sigma'] for psi in scat1d.psi1_f]
sigma2s = [sr * psi['sigma'] for psi in scat1d.psi2_f]

mid_xi1s = np.sqrt(xi1s[1:] * xi1s[:-1])  # center for two valuess in a log scale
mid_xi2s = np.sqrt(xi2s[1:] * xi2s[:-1])
x_coords = np.array([sr//2] + list(mid_xi2s) + [xi2s[-1]-sigma2s[-1]])
y_coords = np.array([sr//2] + list(mid_xi1s) + [xi1s[-1]-sigma1s[-1]])

x_coords = np.hstack((x_coords, x_coords[-1] / 2))

In [219]:
Sx_low_norm = normalize_scattering_data(Sx_low, S.meta(), low)
Sx_high_norm = normalize_scattering_data(Sx_high, S.meta(), high)

(27, 1250)
(27, 1250)


In [220]:
Sx_low_norm.shape

(27, 1250)

In [221]:
Sx_low_r, xi1s,xi2s, order1, order2 = remove_bad_features(J, Q, L, sr=1000, features=Sx_low_norm)
Sx_high_r,_,_,_,_= remove_bad_features(J,Q,L,sr=1000,features=Sx_high_norm)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
(27, 2)
27
[         nan 456.78638314 431.14893343 406.9504032  384.11003212
 362.55159256 332.33895985 302.12632713 271.91369442 241.70106171
 211.48842899 181.27579628 151.06316357 120.85053085  90.63789814
  60.42526543  30.21263271 456.78638314 431.14893343 406.9504032
 384.11003212 362.55159256 332.33895985 302.12632713 271.91369442
 241.70106171 211.48842899]
[  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
   nan   nan   nan   nan   nan 43.75 43.75 43.75 43.75 43.75 43.75 43.75
 43.75 43.75 43.75]
[0, 1, 2, 3, 3, 5, 5, 7, 9, 9, 10, 11, 12, 13, 14, 15, 15, 17, 18, 19, 19, 21, 21, 23, 25, 25, 26]
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
(27, 2)
27
[         nan 456.78638314 431.14893343 406.9504032  384.11003212
 362.55159256 332.33895985 302.12632713 271.91369442 241.70106171
 211.48842899 181.27579628 151.06316357 120.85053085  90.63789814
  60.42526543  30.21263271 456.78638314 431.14893343 406.9

In [222]:
# STAI3520
import matplotlib.pyplot as plt
%matplotlib qt


fig = plt.figure(figsize=(10, 10))
plt.subplot(5, 2, 1)
plt.plot(low)
plt.title('Normalized PCG, Low Stress')
plt.xlim(0, len(low))
plt.xlabel(r'$L$')
plt.ylabel(r'$Amplitude$')

plt.subplot(5, 2, 2)
plt.plot(high)
plt.title('Normalized PCG, High Stress')
plt.xlim(0, len(high))
plt.xlabel(r'$L$')
plt.ylabel(r'$Amplitude$')

plt.subplot(5, 2, 3)
plt.imshow(Sx_low_norm[order1], aspect='auto', cmap='jet')
plt.title(r'$\tilde{S}_1$, Low Stress')
plt.xlabel(r'$L/T$')
plt.ylabel(r'$\lambda1$')

plt.subplot(5, 2, 4)
plt.imshow(Sx_high_norm[order1], aspect='auto', cmap='jet')
plt.title(r'$\tilde{S}_1$, High Stress')
plt.xlabel(r'$L/T$')
plt.ylabel(r'$\lambda1$')

plt.subplot(5, 2, 5)
plt.imshow(Sx_low_norm[order2], aspect='auto', cmap='jet')
plt.title(r'$\tilde{S}_2$, Low Stress')
plt.xlabel(r'$L/T$')
plt.ylabel(r'$\lambda1, \lambda_2$')

plt.subplot(5, 2, 6)
plt.imshow(Sx_high_norm[order2], aspect='auto', cmap='jet')
plt.title(r'$\tilde{S}_2$, High Stress')
plt.xlabel(r'$L/T$')
plt.ylabel(r'$\lambda1, \lambda_2$')

plt.subplot(5, 2, 7)
plt.imshow(Sx_low_norm[xi1s], aspect='auto', cmap='jet')
plt.title(r'$\tilde{S}_1$ after feature exclusion, Low Stress')
plt.xlabel(r'$L/T$')
plt.ylabel(r'$\lambda1$')

plt.subplot(5, 2, 8)
plt.imshow(Sx_high_norm[xi1s], aspect='auto', cmap='jet')
plt.title(r'$\tilde{S}_1$ after feature exclusion, High Stress')
plt.xlabel(r'$L/T$')
plt.ylabel(r'$\lambda1$')

plt.subplot(5, 2, 9)
plt.imshow(Sx_low_norm[xi2s], aspect='auto', cmap='jet')
plt.title(r'$\tilde{S}_2$ after feature exclusion, Low Stress')
plt.xlabel(r'$L/T$')
plt.ylabel(r'$\lambda1, \lambda_2$')

plt.subplot(5, 2, 10)
plt.imshow(Sx_high_norm[xi2s], aspect='auto', cmap='jet')
plt.title(r'$\tilde{S}_2$ after feature exclusion, High Stress')
plt.xlabel(r'$L/T$')
plt.ylabel(r'$\lambda1, \lambda_2$')

plt.tight_layout()
plt.show()

In [223]:
# STAI3520
import matplotlib.pyplot as plt
%matplotlib qt


fig = plt.figure(figsize=(10, 10))
plt.subplot(7, 2, 1)
plt.plot(low)
plt.title('Normalized PCG, Low Stress')
plt.xlim(0, len(low))
plt.xlabel(r'$L$')
plt.ylabel(r'$Amplitude$')

plt.subplot(7, 2, 2)
plt.plot(high)
plt.title('Normalized PCG, High Stress')
plt.xlim(0, len(high))
plt.xlabel(r'$L$')
plt.ylabel(r'$Amplitude$')

plt.subplot(7, 2, 3)
plt.imshow(Sx_low[order1], aspect='auto', cmap='jet')
plt.title(r'$S_1$, Low Stress')
plt.xlabel(r'$L/T$')
plt.ylabel(r'$\lambda1$')

plt.subplot(7, 2, 4)
plt.imshow(Sx_high[order1], aspect='auto', cmap='jet')
plt.title(r'$S_1$, High Stress')
plt.xlabel(r'$L/T$')
plt.ylabel(r'$\lambda1$')

plt.subplot(7, 2, 5)
plt.imshow(Sx_low[order2], aspect='auto', cmap='jet')
plt.title(r'$S_2$, Low Stress')
plt.xlabel(r'$L/T$')
plt.ylabel(r'$\lambda1, \lambda_2$')

plt.subplot(7, 2, 6)
plt.imshow(Sx_high[order2], aspect='auto', cmap='jet')
plt.title(r'$S_2$, High Stress')
plt.xlabel(r'$L/T$')
plt.ylabel(r'$\lambda1, \lambda_2$')

plt.subplot(7, 2, 7)
plt.imshow(Sx_low_norm[order1], aspect='auto', cmap='jet')
plt.title(r'$\tilde{S}_1$, Low Stress')
plt.xlabel(r'$L/T$')
plt.ylabel(r'$\lambda1$')

plt.subplot(7, 2, 8)
plt.imshow(Sx_high_norm[order1], aspect='auto', cmap='jet')
plt.title(r'$\tilde{S}_1$, High Stress')
plt.xlabel(r'$L/T$')
plt.ylabel(r'$\lambda1$')

plt.subplot(7, 2, 9)
plt.imshow(Sx_low_norm[order2], aspect='auto', cmap='jet')
plt.title(r'$\tilde{S}_2$, Low Stress')
plt.xlabel(r'$L/T$')
plt.ylabel(r'$\lambda1, \lambda_2$')

plt.subplot(7, 2, 10)
plt.imshow(Sx_high_norm[order2], aspect='auto', cmap='jet')
plt.title(r'$\tilde{S}_2$, High Stress')
plt.xlabel(r'$L/T$')
plt.ylabel(r'$\lambda1, \lambda_2$')

plt.subplot(7, 2, 11)
plt.imshow(Sx_low_norm[xi1s], aspect='auto', cmap='jet')
#plt.axhline(y=1, color='r', linestyle='--')
#plt.axhline(y=3, color='r', linestyle='--')
plt.title(r'$\tilde{S}_1$ after feature exclusion, Low Stress')
plt.xlabel(r'$L/T$')
plt.ylabel(r'$\lambda1$')

plt.subplot(7, 2, 12)
plt.imshow(Sx_high_norm[xi1s], aspect='auto', cmap='jet')
#plt.axhline(y=1, color='r', linestyle='--')
#plt.axhline(y=3, color='r', linestyle='--')
plt.title(r'$\tilde{S}_1$ after feature exclusion, High Stress')
plt.xlabel(r'$L/T$')
plt.ylabel(r'$\lambda1$')

plt.subplot(7, 2, 13)
plt.imshow(Sx_low_norm[xi2s], aspect='auto', cmap='jet')
#plt.axhline(y=0, color='r', linestyle='--')
#plt.axhline(y=2, color='r', linestyle='--')
plt.title(r'$\tilde{S}_2$ after feature exclusion, Low Stress')
plt.xlabel(r'$L/T$')
plt.ylabel(r'$\lambda1, \lambda_2$')

plt.subplot(7, 2, 14)
plt.imshow(Sx_high_norm[xi2s], aspect='auto', cmap='jet')
#plt.axhline(y=0, color='r', linestyle='--')
#plt.axhline(y=2, color='r', linestyle='--')
plt.title(r'$\tilde{S}_2$ after feature exclusion, High Stress')
plt.xlabel(r'$L/T$')
plt.ylabel(r'$\lambda1, \lambda_2$')

plt.tight_layout()
plt.show()

In [224]:
import matplotlib.pyplot as plt
%matplotlib qt


fig = plt.figure(figsize=(5, 10))
plt.subplot(2, 2, 1)
plt.imshow(Sx_low_norm[xi1s], aspect='auto', cmap='jet')
plt.axhline(y=0, color='green', linestyle='--', label='384.11 Hz')
plt.axhline(y=2, color='r', linestyle='--', label='271.91 Hz')
plt.title(r'$\tilde{S}_1$ after feature exclusion, Low Stress')
plt.xlabel(r'$L/T$')
plt.ylabel(r'$\lambda1$')
plt.legend()

plt.subplot(2, 2, 2)
plt.imshow(Sx_high_norm[xi1s], aspect='auto', cmap='jet')
plt.axhline(y=0, color='green', linestyle='--', label='384.11 Hz')
plt.axhline(y=2, color='r', linestyle='--', label='271.91 Hz')
plt.title(r'$\tilde{S}_1$ after feature exclusion, High Stress')
plt.xlabel(r'$L/T$')
plt.ylabel(r'$\lambda1$')
plt.legend()

plt.subplot(2, 2, 3)
plt.imshow(Sx_low_norm[xi2s], aspect='auto', cmap='jet')
plt.axhline(y=0, color='green', linestyle='--', label='(384.11, 43.75) Hz')
plt.axhline(y=2, color='r', linestyle='--', label='(271.91, 43.75) Hz')
plt.title(r'$\tilde{S}_2$ after feature exclusion, Low Stress')
plt.xlabel(r'$L/T$')
plt.ylabel(r'$\lambda1, \lambda_2$')
plt.legend()

plt.subplot(2, 2, 4)
plt.imshow(Sx_high_norm[xi2s], aspect='auto', cmap='jet')
plt.axhline(y=0, color='green', linestyle='--', label='(384.11, 43.75) Hz')
plt.axhline(y=2, color='r', linestyle='--', label='(271.91, 43.75) Hz')
plt.title(r'$\tilde{S}_2$ after feature exclusion, High Stress')
plt.xlabel(r'$L/T$')
plt.ylabel(r'$\lambda1, \lambda_2$')
plt.legend()

plt.tight_layout()
plt.show()

In [225]:
a = [[1,2],[3,4]]
print(a[:][0])

[1, 2]


In [226]:
np.pad(np.array([[1,2,3],[4,5,6]]),mode='reflect',pad_width=((0,0),(3,3)))

array([[2, 3, 2, 1, 2, 3, 2, 1, 2],
       [5, 6, 5, 4, 5, 6, 5, 4, 5]])

In [227]:
import matplotlib.pyplot as plt
%matplotlib qt

plt.figure(figsize=(10, 8))

plt.subplot(4,2,1)
plt.plot(low)
plt.title('Original Signal, Low Stress')
plt.xlabel(r'$Time$ $[ms]$')

plt.subplot(4, 2, 3)
plt.plot(Sx_low[order0][0])
plt.title(r'$S_0$, Low Stress')
plt.xlabel(r'$Time$ $Averaged$ $[ms/T]$')

plt.subplot(4, 2, 5)
plt.imshow(Sx_low[order1], aspect='auto')
plt.title(r'$S_1$, Low Stress')
plt.xlabel(r'$Time$ $Averaged$ $[ms/T]$')
plt.ylabel(r'$\lambda_1$')

plt.subplot(4, 2, 7)
plt.imshow(Sx_low[order2], aspect='auto')
plt.title(r'$S_2$, Low Stress')
plt.xlabel(r'$Time$ $Averaged$ $[ms/T]$')
plt.ylabel(r'$\lambda_1$, $\lambda_2$')

plt.subplot(4,2,2)
plt.plot(high)
plt.title('Original Signal, High Stress')
plt.xlabel(r'$Time$ $[ms]$')

plt.subplot(4, 2, 4)
plt.plot(Sx_high[order0][0])
plt.title(r'$S_0$, High Stress')
plt.xlabel(r'$Time$ $Averaged$ $[ms/T]$')

plt.subplot(4, 2, 6)
plt.imshow(Sx_high[order1], aspect='auto')
plt.title(r'$S_1$, High Stress')
plt.xlabel(r'$Time$ $Averaged$ $[ms/T]$')
plt.ylabel(r'$\lambda_1$')

plt.subplot(4, 2, 8)
plt.imshow(Sx_high[order2], aspect='auto')
plt.title(r'$S_2$, High Stress')
plt.xlabel(r'$Time$ $Averaged$ $[ms/T]$')
plt.ylabel(r'$\lambda_1$, $\lambda_2$')

plt.tight_layout()
plt.show()


In [228]:
import matplotlib.pyplot as plt
%matplotlib qt

plt.figure(figsize=(15, 8))

plt.subplot(3,2,1)
plt.plot(low)
plt.title('Original Signal, Low Stress')
plt.xlabel(r'$Time$ $[ms]$')

plt.subplot(3, 2, 3)
plt.imshow(Sx_low[order1], aspect='auto')
plt.title(r'$S_1$, Low Stress')
plt.xlabel(r'$Time$ $Averaged$ $[ms/T]$')
plt.ylabel(r'$\lambda_1$')

plt.subplot(3, 2, 5)
plt.imshow(Sx_low[order2], aspect='auto')
plt.title(r'$S_2$, Low Stress')
plt.xlabel(r'$Time$ $Averaged$ $[ms/T]$')
plt.ylabel(r'$\lambda_1$, $\lambda_2$')

plt.subplot(3,2,2)
plt.plot(high)
plt.title('Original Signal, High Stress')
plt.xlabel(r'$Time$ $[ms]$')

plt.subplot(3, 2, 4)
plt.imshow(Sx_high[order1], aspect='auto')
plt.title(r'$S_1$, High Stress')
plt.xlabel(r'$Time$ $Averaged$ $[ms/T]$')
plt.ylabel(r'$\lambda_1$')

plt.subplot(3, 2, 6)
plt.imshow(Sx_high[order2], aspect='auto')
plt.title(r'$S_2$, High Stress')
plt.xlabel(r'$Time$ $Averaged$ $[ms/T]$')
plt.ylabel(r'$\lambda_1$, $\lambda_2$')

plt.tight_layout()
plt.show()

BACKGROUND AND THEORY EXAMPLE

In [229]:
import numpy as np
from kymatio import Scattering1D

J = 8
Q = 10
fs = 1000
sec = 16
L = int(np.floor(fs*sec/2**J))*2**J

i = 10
S = Scattering1D(J=J, shape=L, Q=Q, max_order=2)

low, high = d['low'][L*i:L*(i+1)], d['high'][L*i:L*(i+1)]
print(L, len(low))
Sx_low = S(low)
Sx_high = S(high)
print(Sx_low.shape, Sx_high.shape)

meta = S.meta()
order0 = np.where(meta['order'] == 0)
order1 = np.where(meta['order'] == 1)
order2 = np.where(meta['order'] == 2)

print(Sx_low[order0].shape, Sx_low[order1].shape, Sx_low[order2].shape)
print(order1)

15872 15872
(286, 62) (286, 62)
(1, 62) (65, 62) (220, 62)
(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65],
      dtype=int64),)


In [230]:
xi1s = np.array([sr * psi['xi'] for psi in scat1d.psi1_f])
xi2s = np.array([sr * psi['xi'] for psi in scat1d.psi2_f])

sigma1s = [sr * psi['sigma'] for psi in scat1d.psi1_f]
sigma2s = [sr * psi['sigma'] for psi in scat1d.psi2_f]

mid_xi1s = np.sqrt(xi1s[1:] * xi1s[:-1])  # center for two valuess in a log scale
mid_xi2s = np.sqrt(xi2s[1:] * xi2s[:-1])
x_coords = np.array([sr//2] + list(mid_xi2s) + [xi2s[-1]-sigma2s[-1]])
y_coords = np.array([sr//2] + list(mid_xi1s) + [xi1s[-1]-sigma1s[-1]])

x_coords = np.hstack((x_coords, x_coords[-1] / 2))

In [231]:
# scattering1d config
kwargs = {
    'shape': 2**14,
    'J': 8,
    'T': 2**8,
    'Q': (10, 1)}

scat1d = Scattering1D(**kwargs)
meta = scat1d.meta()
sr = 1000
freq_min = 0

# carrier center frequencies condition >= freq_min
freq_cnd = meta['xi'][:, 0] * sr >= freq_min

# indices for scat coefficient to extract 
idx_S1 = np.where(np.logical_and(meta['order'] == 1, freq_cnd))[0]  # order 1
idx_S2 = np.where(np.logical_and(meta['order'] == 2, freq_cnd))[0] # order 2
idx_S1S2 = np.where(np.logical_and(meta['order'] != 0, freq_cnd))[0] # order 1 and 2

In [232]:
Sx_low_norm = normalize_scattering_data(Sx_low, S.meta(), low)
Sx_high_norm = normalize_scattering_data(Sx_high, S.meta(), high)

(286, 62)
(286, 62)


In [233]:
Sx_low_r, xi1s,xi2s, order1, order2 = remove_bad_features(J, Q, L, sr=1000, features=Sx_low_norm)
Sx_high_r,_,_,_,_= remove_bad_features(J,Q,L,sr=1000,features=Sx_high_norm)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65]
(286, 2)
286
[         nan 448.20048134 418.18583591 390.1811815  364.05191502
 339.67244734 316.92559969 295.70204037 275.89975933 257.42357781
 240.1846909  224.10024067 209.09291796 195.09059075 182.02595751
 169.83622367 158.46279984 147.85102019 137.94987967 128.71178891
 120.09234545 112.05012033 104.54645898  97.54529538  91.01297875
  84.91811184  79.23139992  73.92551009  68.97493983  64.35589445
  60.04617272  56.02506017  52.27322949  48.77264769  45.50648938
  42.45905592  39.61569996  36.96275505  34.48746992  32.17794723
  30.02308636  28.01253008  26.13661474  24.38632384  22.75324469
  21.22952796  19.80784998  18.48137752  17.24373496  16.08897361
  15.01154318  14.00626504  13.06830737  12.19316192  11.37662234
  10.61476398   9.90392499   8.91353249   7.92313999   6.93274

In [234]:
import matplotlib.pyplot as plt
%matplotlib qt

plt.figure(figsize=(15, 8))

plt.subplot(4,2,1)
plt.plot(low)
plt.title('Original Signal, Low Stress')
plt.xlabel(r'$Time$ $[ms]$')
plt.ylabel('Amplitude')

plt.subplot(4, 2, 3)
plt.plot(Sx_low[0])
plt.title(r'$S_0$, Low Stress')
plt.xlabel(r'$Time$ $Averaged$ $[ms/T]$')
plt.ylabel('Amplitude')

plt.subplot(4, 2, 5)
plt.imshow(Sx_low[order1], aspect='auto')
plt.title(r'$S_1$, Low Stress')
plt.xlabel(r'$Time$ $Averaged$ $[ms/T]$')
plt.ylabel(r'$\lambda_1$')

plt.subplot(4, 2, 7)
plt.imshow(Sx_low[order2], aspect='auto')
plt.title(r'$S_2$, Low Stress')
plt.xlabel(r'$Time$ $Averaged$ $[ms/T]$')
plt.ylabel(r'$\lambda_1$, $\lambda_2$')

plt.subplot(4,2,2)
plt.plot(high)
plt.title('Original Signal, High Stress')
plt.xlabel(r'$Time$ $[ms]$')
plt.ylabel('Amplitude')

plt.subplot(4, 2, 4)
plt.plot(Sx_high[0])
plt.title(r'$S_0$, High Stress')
plt.xlabel(r'$Time$ $Averaged$ $[ms/T]$')
plt.ylabel('Amplitude')


plt.subplot(4, 2, 6)
plt.imshow(Sx_high[order1], aspect='auto')
plt.title(r'$S_1$, High Stress')
plt.xlabel(r'$Time$ $Averaged$ $[ms/T]$')
plt.ylabel(r'$\lambda_1$')

plt.subplot(4, 2, 8)
plt.imshow(Sx_high[order2], aspect='auto')
plt.title(r'$S_2$, High Stress')
plt.xlabel(r'$Time$ $Averaged$ $[ms/T]$')
plt.ylabel(r'$\lambda_1$, $\lambda_2$')

plt.tight_layout()
plt.show()